In [2]:
import os
import itertools
import numpy as np
import pandas as pd
import cv2
from scipy.spatial.distance import cdist,euclidean
from numpy import arccos, array, dot, pi, cross
from numpy.linalg import det, norm
from numpy import array_equal, allclose
import sys
!{sys.executable} -m pip install tabulate
from tabulate import tabulate


kernel = np.ones((2,2),np.uint8)
anchor = (320,250)


""" Function to calculate the distance between a line segment and a point """
def distance_lp(A, B, P):
    if all(A == P) or all(B == P):
        return 0
    if arccos(np.dot((P - A) / norm(P - A), (B - A) / norm(B - A))) > pi / 2:
        return norm(P - A)
    if arccos(np.dot((P - B) / norm(P - B), (A - B) / norm(A - B))) > pi / 2:
        return norm(P - B)
    return norm(cross(A-B, A-P))/norm(B-A)


""" Function to calculate the closest point to a given point B from an array of points """
def closest_node(node, nodes):
    return nodes[cdist([node], nodes).argmin()]


""" Function to determine where a point lies wrt to a line segment. Returns 0 if on it, +1 or -1 if not, depending on the side """
def line_side(A,B,Pt):
    return np.sign(np.cross(array([A[0] - B[0],A[1] - B[1]]),array([A[0] - Pt[0],A[1] - Pt[1]])))


""" Function to check whether two points lie on different sides of a line segment. Returns TRUE if they are on different sides """
def line_side_check(A,B,anc,P):
    if abs(line_side(A,B,anc)) + abs(line_side(A,B,P)) != abs(line_side(A,B,anc) + line_side(A,B,P)):
        return True
    else:
        return False

    
""" Function to check whether a list of arrays contains an array of interest """
def arreq_in_list(myarr, list_arrays):
    return next((True for elem in list_arrays if array_equal(elem, myarr)), False)


""" Function to calculate the shortest distance between two line segments """
def shortest_distance_ll(segment1,segment2):
    A1 = segment1[0]
    B1 = segment1[1]
    A2 = segment2[0]
    B2 = segment2[1]
    A12 = distance_lp(A2,B2,A1)
    B12 = distance_lp(A2,B2,B1)
    A21 = distance_lp(A1,B1,A2)
    B21 = distance_lp(A1,B1,B2)
    return min(A12,B12,A21,B21)


""" Function to check whether a point lies in the FIELD OF INTEREST of a line segment """
def shortest_dp(segment1,P):
    XA = segment1[0]
    XB = segment1[1]
    if (arccos(np.dot((P-XA) / norm(P-XA), (XB-XA) / norm(XB-XA))) <= (pi)/2) and (arccos(np.dot((P-XB) / norm(P-XB), (XA-XB) / norm(XA-XB))) <= (pi)/2):
        return True
    else:
        return False


""" Function to calculate distance from Anchor point to the building or road"""

def dist_from_anch(x,y,z):
    
    rows, cols = np.shape(x)
    for i in range(rows):                                                  # THRESHOLDING AND PERFORMING MORPHOLOGICAL OPS FOR 
        for j in range(cols):                                              # EXTRACTING THE CONTOURS OF BUILDINGS
            if (x[i][j] >= 249 and x[i][j] <= 253) or (x[i][j] > 240 and x[i][j] < 245):
                x[i][j] = 255
            else:
                x[i][j] = 0
                                                                           # FINDING AND STORING ALL CO-ORDS OF EACH CONTOUR
    opening = cv2.morphologyEx(x, cv2.MORPH_ERODE, kernel)                   
    contours_y, hierarchy_y = cv2.findContours(opening,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)    



    rows, cols = np.shape(y)                                               # EXTRACTING THE CONTOURS OF ROADS AFTER THRESHOLDING
    for i1 in range(rows):
        for j1 in range(cols):
            if y[i1][j1] > 253:
                y[i1][j1] = 255                                            # SEPARATELY EXTRACTED ROAD AND BUILDING CONTOURS TO AVOID INTERSECTION
            else:
                y[i1][j1] = 0

    contours_r, hierarchy_r = cv2.findContours(y,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)


    contours_b = []                                                         # MERGING THE TWO INTO ONE GLOBAL CONTOUR LIST
    for i2 in contours_y:
        contours_b.append(i2)
    for j2 in contours_r:
        contours_b.append(j2)



    cpoints = []                                                       # EXTRACTING THE CORNER POINTS OF EACH ONE OF THE CONTOUR.
    recont = [] 
    for cnt in contours_b:
        mad = np.full(np.shape(x),255)
        mad = cv2.UMat(mad)
        if cv2.contourArea(cnt)>50:
#             cv2.drawContours()
            cv2.drawContours(mad,cnt,-1,0,1)
            mad = cv2.UMat.get(mad)                               # DO NOT USE UMAT object - THATS CAUSING THE NONETYPE ERROR - extract np array
            mad = np.float32(mad)
            corners = cv2.goodFeaturesToTrack(mad,maxCorners = 5,qualityLevel = 0.01, minDistance = 20, blockSize = 3)
            corners = np.int0(corners)
            ttt = np.vstack(cnt).squeeze()
            ncorners = []
            for i in corners:
                b = i.ravel()
                ncorners.append(closest_node(b,ttt))                   # APPROXIMATING THE CORNERS TO NEAREST CONTOUR POINTS FOR ACCURACY
            cpoints.append(ncorners)
            recont.append(cnt)

    
    del contours_b
    del mad
    del corners
    del ncorners


    cont_dist = []
    for i in range(len(recont)):
        cont_dist.append(cv2.pointPolygonTest(recont[i],anchor,True))   # CHECKING WHETHER THE ANCHOR LIES IN/ON A CONTOUR OR NOT
        idx = cont_dist.index(max(cont_dist))
    if arreq_in_list(recont[idx], contours_y):
        flag = 'Building'
    elif arreq_in_list(recont[idx], contours_r):
        flag = 'Road'


    ccc = np.vstack(recont[idx]).squeeze()    # ESTIMATING THE POINT CLOSEST TO ANCHOR POINT ON THE CONTOUR
    cln = closest_node(anchor,ccc)
    
    del ccc


    if max(cont_dist) < 0:
        output_flag = flag
        smallest_distance_cln = round(abs(max(cont_dist)))
        smallest_distance_walls = smallest_distance_cln
    else:
        if flag == 'Road':
            smallest_distance_cln = round(abs(sorted(cont_dist)[-2]))
            smallest_distance_walls = smallest_distance_cln
            output_flag = 'Building'

        if flag == 'Building':
            combos = []                              # ESTIMATING THE EDGE CLOSEST TO ANCHOR POINT
            for pt in cpoints[idx]:                  # (CAN BE IDENTIFIED MORE ACCURATELY USING 'cln')
                 combos.append(pt)
            min_dist = 10000
            for cmb in itertools.combinations(combos,2):
                A = cmb[0]
                B = cmb[1]
                dist1 = distance_lp(A,B,cln)
                if dist1 < min_dist:
                    min_dist = dist1
                    segment = cmb
            b_cont = []                             # IDENTIFYING CONTOURS AND THEIR CORNERS THAT LIE IN THE REGION OF INTEREST
            b_cpoints = []
            r_cont = []
            for c in range(len(recont)):
                if arreq_in_list(recont[c], contours_y):                   # FINDING EXTREME POINTS ONLY FOR BUILDINGS    
                    extLeft = recont[c][recont[c][:, :, 0].argmin()][0]     
                    extRight = recont[c][recont[c][:, :, 0].argmax()][0]    # IDENTIFICATION SIMPLIFIED BY USING ONLY EXTREME POINTS
                    extTop = recont[c][recont[c][:, :, 1].argmin()][0]      # OF THE CONTOUR
                    extBot = recont[c][recont[c][:, :, 1].argmax()][0]

                    if(line_side_check(segment[0],segment[1],anchor,extLeft) == True) and (line_side_check(segment[0],segment[1],anchor,extRight) == True) and (line_side_check(segment[0],segment[1],anchor,extTop) == True) and (line_side_check(segment[0],segment[1],anchor,extBot) == True):
                        b_cont.append(recont[c])
                        b_cpoints.append(cpoints[c])

            closest_cont_distance = []
#             a = []
            cont_cln_distance = []
            for nc in range(len(b_cont)):
                combi = []
                for pts in b_cpoints[nc]:
                    combi.append(pts)                                   # EXTRACTING THE SHORTEST DISTANCES FROM EACH BUILDING TO
                how_far = 11000                                         # THE SHOP'S WALL. DISTANCE MEASURED FROM EACH ONE
                a1 = (_,_)
                cln_dist = 10000                                        # OF THE EDGES OF EVERY BUILDING - CHOSE MINIMUM.
                for cmbi in itertools.combinations(combi,2):    
                    if shortest_distance_ll(segment,cmbi) < how_far:
                        if(shortest_dp(cmbi,segment[0]) == True) or (shortest_dp(cmbi,segment[1]) == True) or (shortest_dp(segment,cmbi[0]) == True) or (shortest_dp(segment,cmbi[1]) == True):
                            how_far = shortest_distance_ll(segment,cmbi)
                            cln_dist = distance_lp(cmbi[0],cmbi[1],cln)
#                             a1 = cmbi
                    
                closest_cont_distance.append(how_far)
                cont_cln_distance.append(cln_dist)
#                 a.append(a1)

#             closest_wall = a[closest_cont_distance.index(min(closest_cont_distance))]
#             closest_building = b_cont[closest_cont_distance.index(min(closest_cont_distance))]                           # THE CLOSEST CONTOUR, AND ITS DISTANCE FROM THE WALL OF BUILDING
            smallest_distance_walls = round(min(closest_cont_distance))                                                  # ARE EXTRACTED
            smallest_distance_cln = round(cont_cln_distance[closest_cont_distance.index(min(closest_cont_distance))])   
            output_flag = 'Building'

            xc = np.vstack(contours_r).squeeze()
            closest_pt_rd = closest_node(cln,xc)
            if (shortest_dp(segment,closest_pt_rd) == True) and (line_side_check(segment[0],segment[1],anchor,closest_pt_rd) == True):
                shtdist_walls = distance_lp(segment[0],segment[1],closest_pt_rd)
                shtdist_cln = euclidean(cln,closest_pt_rd)
                if shtdist_walls < smallest_distance_walls:
                    smallest_distance_walls = round(shtdist_walls)
                if shtdist_cln < smallest_distance_cln:
                    smallest_distance_cln = round(shtdist_cln)
                    output_flag = 'Road'
        
    table = [z,  smallest_distance_walls, output_flag, anchor]
    return table
    
    

if __name__ == "__main__":

    op = []
    path = input('Enter path to your folder')
    valid_images = (".png")
    for f in os.listdir(path):
        if f.lower().endswith(valid_images):
            a1 = (cv2.imread(os.path.join(path,f),0))
            a2 = (cv2.imread(os.path.join(path,f),0))
            op.append(dist_from_anch(a1,a2,os.path.splitext(f)[0]))
    print('\n',tabulate(op, headers=['Name', 'Distance', 'Flag', 'Anchor point']))


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Enter path to your folderC:\Users\sidew\Desktop\av\New folder

 Name      Distance  Flag      Anchor point
------  ----------  --------  --------------
tmp_1           55  Building  (320, 250)
tmp_2           15  Building  (320, 250)
tmp_3           34  Building  (320, 250)
tmp_4           13  Building  (320, 250)
tmp_5          184  Building  (320, 250)
tmp_6           15  Road      (320, 250)
tmp_7            9  Road      (320, 250)
tmp_8           41  Building  (320, 250)
tmp_9           20  Road      (320, 250)
